## 问财分钟模板

In [ ]:
# 爱问财策略回测框架（分钟级回测）
# 问财分钟模板
import datetime

# 初始化账户
def init(context):
    # 设置策略问句
    get_iwencai('非停牌;非st;业绩预增大于50%；市值排名后30%；市盈率小于120；总市值从小到大')
    # 设置持仓股票数量
    context.n = 4
    # 设置股票持有期
    context.holding_period = 5
    # 设置止盈涨幅
    context.stop_gain = 0.2
    # 设置止盈回撤
    context.stop_gain_drawdown = 0.05
    # 设置止损跌幅
    context.stop_loss = 0.05
    # 设置仓位比例
    context.position_pct = 1
    
    context.information = {}
    
    run_daily(market_open,'after_open',0,1,'000001.SZ')
    run_daily(market_close,'before_close',0,5,'000001.SZ')
    

class Information:
    # 记录买入股票的信息，作为position的补充
    def __init__(self, remain_days):
        self.remain_days = remain_days
        self.stop_gain = False
        self.today_high = 0

# 开盘时买入股票
def market_open(context, bar_dict):
    # 若持有天数到期则卖出（防止昨天未卖出）
    for stock in list(context.portfolio.positions):
        context.information[stock].today_high = 0
        if (not context.information[stock].stop_gain
                and context.information[stock].remain_days == 0):
            order_target(stock, 0)
            if stock not in list(context.portfolio.positions):
                del context.information[stock]
                log.info('持有天数到期，卖出%s' % stock)
                continue
    
    stocks_list = context.iwencai_securities
    number_of_buys = context.n-len(context.portfolio.positions)
    if number_of_buys == 0:
        return
    # 若持有股数不足设定值则买入
    cash_for_stock = context.portfolio.available_cash*context.position_pct/number_of_buys
    
    for stock in stocks_list:
        if stock in context.portfolio.positions:
            continue
        #current = data.current([stock])[stock]
        if not bar_dict[stock].open or not bar_dict[stock].close:
            continue
        # 开盘涨跌幅不小于-9%才买入
        if bar_dict[stock].open/bar_dict[stock].close-1 >= -0.09:
            order_value(stock, cash_for_stock)
            if stock in list(context.portfolio.positions):
                context.information[stock] = Information(context.holding_period)
                log.info('买入%s' % stock)
        number_of_buys -= 1
        if number_of_buys == 0:
            return

# 盘中止盈止损
def handle_bar(context, bar_dict):
    price_dict = history(list(context.portfolio.positions), ['close'], 2, '1m')
    for stock in price_dict:
        if price_dict[stock].empty:
            continue
        price = price_dict[stock].values[-1][0]
        # 更新今日最高价
        if price > context.information[stock].today_high:
            context.information[stock].today_high = price
        # 若刚买入则不进行操作
        if context.information[stock].remain_days == context.holding_period:
            continue
        # 若进入止盈状态，则根据要求止盈
        if context.information[stock].stop_gain:
            if price/context.information[stock].high-1 < -context.stop_gain_drawdown:
                order_target(stock, 0)
                if stock not in context.portfolio.positions:
                    del context.information[stock]
                    log.info('止盈，卖出%s' % stock)
        # 若未进入止盈状态，则根据要求止损
        else:
            if price/context.portfolio.positions[stock].cost_basis-1 < -context.stop_loss:
                order_target(stock, 0)
                if stock not in context.portfolio.positions:
                    del context.information[stock]
                    log.info('止损，卖出%s' % stock)

# 收盘时卖出到期股票、判断止盈
def market_close(context, bar_dict):
    for stock in list(context.portfolio.positions):
        today_high = context.information[stock].today_high
        # 若进入止盈状态，则更新最高价
        if context.information[stock].stop_gain:
            if today_high > context.information[stock].high:
                context.information[stock].high = today_high
        # 若未进入止盈状态，则卖出到期股票、判断止盈
        else:
            # 判断是否进入止盈状态
            if today_high/context.portfolio.positions[stock].cost_basis-1 >= context.stop_gain:
                context.information[stock].stop_gain = True
                context.information[stock].high = today_high
                log.info('%s进入止盈状态' % stock)
                continue
            # 若持有天数到期则卖出
            context.information[stock].remain_days -= 1
            if context.information[stock].remain_days == 0:
                order_target(stock, 0)
                if stock not in context.portfolio.positions:
                    del context.information[stock]
                    log.info('持有天数到期，卖出%s' % stock)
                    continue
                    